In [11]:
import sys,os
import xarray as xr
import numpy as np
import scipy as spy
import pickle 

import matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
from collections import OrderedDict 

import time as timestamp 

In [12]:
sys.path.append("/home/disk/p/mkb22/Documents/si_analysis_kb/LIMs/SI_LIMs/")
import LIM_utils as lim

import LIM_utils_kb as limkb
import LIM_stats_kb as statskb
import LIM_plot_kb as plotkb
import LIM_building as limbuild

In [48]:
import importlib
importlib.reload(limkb)
importlib.reload(statskb)
importlib.reload(limbuild)

<module 'LIM_building' from '/home/disk/p/mkb22/Documents/si_analysis_kb/LIMs/SI_LIMs/LIM_building.py'>

In [14]:
arc_proj = dict(projection=ccrs.Stereographic(central_latitude=90,
                                              central_longitude=-45,
                                              true_scale_latitude=0.1))
proj = dict(projection=ccrs.Robinson(central_longitude=0.),zorder=1)

In [15]:
month_names = ['Jan','Feb','Mar','Apr','May','June','July','Aug','Sept','Oct','Nov','Dec']

In [16]:
from datetime import date

today = date.today()

#Year-month-day
today_date = today.strftime("%Y%m%d")

In [17]:
import warnings
warnings.filterwarnings("ignore")

# Train one month LIM: 

In [ ]:
# number of EOFs to retain for the LIM state vector
ntrunc = 30 # EOF truncation for individual fields (reduced-state space)
nmodes = 30 # number of coupled EOFs for LIM state space (truncated-state space)
#nmodes = 25
nmodes_sic = 50
#modes_sic = 20

#mo='all'
mo=4

# forecast lead time in months that defines the LIM training
tau = 1

# training data defined by the first ntrain times
# fraction of years used in training
# ensures that start year of both train and validation data is january 
ntrain = 1
ntraintimes = 1980
# nvalidtimes = 1872
nttrain_valid = 0
tscut_era = False    # time start cut 
tecut_era = False       # time end cut 

tscut_sat = False    # time start cut 
tecut_sat = 0       # time end cut 

# variables to include in the LIM (note "vars" is a Python command)
#limvars = ['tas','zg']
#limvars = ['tas','rlut','zg']
#limvars = ['tas','tos','psl','sic']
limvars = ['sic']
#limvars = ['tas','sic']
#limvars = ['tas','sic','zg','psl','pr','tos']
#limvars = ['tas','psl','tos','sit','sic']
# limvars = ['tas','tos','psl','sit','sic']
# limvars_nosic = ['tas','tos','psl','sit']
#limvars = ['tas','tos','psl','sit','sic']
#limvars = ['tas','tos','sic']
#limvars_nosic = ['tas','tos']
limvars_nosic = []
nvars = len(limvars)

# specify the model source 
#train_dsource = 'ccsm4_hist_kb'
#train_dsource = 'mpi_hist_kb'
#train_dsource = 'cmip6_cesm2_ssp585'
#train_dsource = 'cmip6_mpi_hist'
train_dsource = 'mpi_lm_kb'
#train_dsource = 'era5'
valid_dsource = 'era5'
#valid_dsource = 'mpi_lm_kb'
#valid_sat = 'satellite'
#valid_dsource = 'ccsm4_lm_kb'
#valid_dsource = 'cmip6_mpi_hist'

sic_separate = True
Insamp = False

In [ ]:
# Start with truncated training data: 
#mod_folder = 'truncated_model_data/last_millennium/'
#mod_folder = 'truncated_model_data/'#+folder_add
mod_folder = ''#+folder_add
#mod_filename = '_ntrunc'+str(ntrunc)+'_month'+str(mo)+'_'+train_dsource+'_'+'ntrain_1850_2004_20210921.pkl'
mod_filename = '_ntrunc'+str(ntrunc)+'_month'+str(mo)+'_'+train_dsource+'_grid05_ntrain_1979_2004_20210914.pkl'

#mod_sic_filename = '_ntrunc'+str(nmodes_sic)+'_month'+str(mo)+'_'+train_dsource+'_ntrain_1850_2050_20210916.pkl'
mod_sic_filename = '_ntrunc'+str(nmodes_sic)+'_month'+str(mo)+'_'+train_dsource+'_grid05_ntrain_1979_2004_20210921.pkl'
#mod_sic_filename = '_ntrunc'+str(nmodes_sic)+'_month'+str(mo)+'_'+train_dsource+'_'+'ntrain_1850_2004_20210921.pkl'
mod_sic_filename = '_ntrunc50_month4_mpi_lm_kb_ntrain_850_1833_20210921.pkl'

In [ ]:
exp_setup = {}
exp_setup['mo'] = mo
if 'all' in str(mo): 
    exp_setup['ind_month_trunc'] = False
else: 
    exp_setup['ind_month_trunc'] = True
exp_setup['tau'] = tau
exp_setup['ntrunc'] = ntrunc 
exp_setup['nmodes_sic'] = nmodes_sic
exp_setup['limvars'] = limvars
exp_setup['train_dsource'] = train_dsource
exp_setup['valid_dsource'] = valid_dsource 
exp_setup['sic_separate'] = sic_separate
exp_setup['Insamp'] = Insamp
exp_setup['mod_folder'] = mod_folder
exp_setup['mod_filename'] = mod_filename
exp_setup['mod_sic_filename'] = mod_sic_filename
exp_setup['Weight']=True
# era5 settings: 
# exp_setup['nyearsvalid'] = 16
# exp_setup['nyearstot'] = 42
# exp_setup['nyears_startvalid'] = 26*12

# Satellite settings: 
# exp_setup['nyearsvalid'] = 12
# exp_setup['nyearstot'] = 38
# exp_setup['nyears_startvalid'] = 26*12

# Historical settings: 
# exp_setup['nyearsvalid'] = 11
# exp_setup['nyearstot'] = 164
# exp_setup['nyears_startvalid'] = 154*12

# LM settings
exp_setup['nyearsvalid'] = 16
exp_setup['nyearstot'] = 1000
exp_setup['nyears_startvalid'] = (1000-16)*12

exp_setup['ntrain']=((exp_setup['nyearstot']*12)-(exp_setup['nyearsvalid'] *12))/(exp_setup['nyearstot']*12)

In [47]:
importlib.reload(rf)
importlib.reload(limbuild)
importlib.reload(limkb)
importlib.reload(statskb)

NameError: name 'rf' is not defined

In [ ]:
import run_forecast_model_data as rf

In [ ]:
[Ptrunc, _, E3, tot_var, 
 tot_var_eig, W_all, 
 standard_factor, 
 nyears_train, var_dict] = load_training_data_truncated(exp_setup['limvars'], exp_setup['mod_folder'], 
                                                        exp_setup['mod_sic_filename'], 
                                                        exp_setup['mod_filename'], 
                                                        exp_setup['mo'],exp_setup['nyearstot'],
                                                        exp_setup['nyearsvalid'],
                                                        ind_month_trunc=exp_setup['ind_month_trunc'])

In [ ]:
var_dict = limbuild.get_var_indices(exp_setup['limvars'], var_dict)

ndof_all = limkb.count_ndof_all(exp_setup['limvars'], E3, sic_separate=exp_setup['sic_separate'])


if len(exp_setup['limvars'])<=1:
    print('Only one variable detected...')
    Ptrunc_all = []
    E3_all = []
    Ptrunc_sic = Ptrunc['sic']
    E_sic = E3['sic']

    P_train = Ptrunc_sic
else: 
    print('Multiple variables detected...')
    [Ptrunc_all, E3_all, 
    Ptrunc_sic,E_sic] = limkb.stack_variable_eofs(exp_setup['limvars'], ndof_all, 
                                                  exp_setup['ntrunc'], Ptrunc, E3,var_dict,
                                                  sic_separate=exp_setup['sic_separate'])

    P_train = np.concatenate((Ptrunc_all, Ptrunc_sic),axis=0)

In [ ]:
# TRAIN LIM: 
    #--------------------------------------------------

nmo = int(P_train.shape[1]/nyears_train)
# nmo = 2
P_train_3d = np.reshape(P_train, (P_train.shape[0],nyears_train,nmo))

if exp_setup['mo'] is 'all':
    LIMd2, G2 = lim.LIM_train(exp_setup['tau'],P_train)
    print('Training LIM with tau = '+str(exp_setup['tau']))
else: 
    LIMd2, G2 = lim.LIM_train_flex(exp_setup['tau'],P_train_3d[:,:,0], P_train_3d[:,:,1])
    print('Training LIM with tau = '+str(exp_setup['tau']))

In [ ]:
#--------------------------------------------------

max_eigenval = np.real(LIMd2['lam_L']).max()

if max_eigenval >0: 
    LIMd2['lam_L_adj'] = LIMd2['lam_L'] - (max_eigenval+0.01)
else: 
    LIMd2['lam_L_adj'] = LIMd2['lam_L']

LIMd2['npos_eigenvalues'] = (LIMd2['lam_L']>0).sum()/(LIMd2['lam_L'].shape[0])
print('Number of positive eigenvalues = '+ str((LIMd2['lam_L']>0).sum()/(LIMd2['lam_L'].shape[0])))

In [ ]:
plt.plot(P_train_3d[3,:,0])
plt.plot(P_train_3d[3,:,1])
plt.xlim(0,100)

In [ ]:
LIMd2['lam_L']

# Train LIM with LME: 

In [53]:
dirname = '/home/disk/chaos/mkb22/Documents/SeaIceData/LME/'
#mnames = ['01','02','03','04','05','06','07','08','09','10','11','12']
mnames = ['01','02']
files_list = [dirname+'sic_global_'+i+'_CESM_CAM5_LME_085001-184912.nc' for i in mnames]

limvars = ['sic']

In [57]:
# number of EOFs to retain for the LIM state vector
ntrunc = 30 # EOF truncation for individual fields (reduced-state space)
nmodes = 30 # number of coupled EOFs for LIM state space (truncated-state space)
#nmodes = 25
nmodes_sic = 50
#modes_sic = 20

mo='all'
#mo=0

# forecast lead time in months that defines the LIM training
tau = 1

# training data defined by the first ntrain times
# fraction of years used in training
# ensures that start year of both train and validation data is january 
ntrain = 1
ntraintimes = 1980
# nvalidtimes = 1872
nttrain_valid = 0
tscut1 = False    # time start cut 
tecut1 = False       # time end cut 

tscut2 = False    # time start cut 
tecut2 = False       # time end cut 

# variables to include in the LIM (note "vars" is a Python command)
#limvars = ['tas','zg']
#limvars = ['tas','rlut','zg']
limvars = ['sic']
#limvars = ['tas','sic']
#limvars = ['tas','sic','zg','psl','pr','tos']
#limvars = ['tas','psl','tos','sit','sic']
# limvars = ['tas','tos','psl','sit','sic']
# limvars_nosic = ['tas','tos','psl','sit']
#limvars = ['tas','tos','psl','sit','sic']
#limvars = ['tas','tos','sic']
#limvars_nosic = ['tas','tos']
limvars_nosic = []
nvars = len(limvars)

# specify the model source 
#train_dsource = 'ccsm4_hist_kb'
#train_dsource = 'mpi_hist_kb'
train_dsource1 = 'cesm_lme_lm_01'

train_dsource2 = 'cesm_lme_lm_02'

sic_separate = True
Insamp = False

In [58]:
infile_20cr_tas = '/home/disk/kalman3/rtardif/LMR/data/model/20cr/tas_sfc_Amon_20CR_185101-201112.nc'

fdic_train1 = limkb.build_training_dic(train_dsource1)
fdic_train2 = limkb.build_training_dic(train_dsource2)
#fdic_valid = limkb.build_training_dic(valid_dsource)

full_names, areawt_name, month_names = limbuild.load_full_names()
areacell, areacell_dict = limbuild.load_areacell_dict(fdic_train1, remove_climo=False, 
                                                      detrend=False, verbose=False )

Loading from os/mkb22/Documents/SeaIceData/LME/areacello_global_LME_01.nc


In [60]:
wt=True
# var_dict1 = {}
# var_dict2 = {}

for k, var in enumerate(limvars):
#     X_var1, var_dict1 = limkb.load_data(var, var_dict1, fdic_train1, remove_climo=True, 
#                                       detrend=True, atol=False, verbose=True, cmip6=False, 
#                                       tscut=tscut1, tecut=tecut1)
    
#     X_var2, var_dict2 = limkb.load_data(var, var_dict2, fdic_train2, remove_climo=True, 
#                                         detrend=True, atol=False, verbose=True, cmip6=False, 
#                                         tscut=tscut2, tecut=tecut2)
    
    if var is 'sic':
        if np.nanmax(X_var1)>1:
            print('Changing units of ds1 sic be a between 0 to 1')
            X_var1 = X_var1/100
        if np.nanmax(X_var2)>1:
            print('Changing units of ds2 sic be a between 0 to 1')
            X_var2 = X_var2/100
    
    X_var = np.concatenate((X_var1,X_var2),axis=1)
    
    tsamp = X_var.shape[1]
    
    acell = areacell[areawt_name[var]]
    if len(acell.shape)>1:
        acell_1d = np.reshape(acell,(acell.shape[0]*acell.shape[1]))
    else: 
        acell_1d = acell
        
    if 'km' in areacell_dict[areawt_name[var]][areawt_name[var]]['units']:
        acell_1d = acell_1d
    else: 
        print('changing cellarea units from '+
              str(areacell_dict[areawt_name[var]][areawt_name[var]]['units'])+' to km^2')
        acell_1d = acell_1d/(1000*1000)
     
    [Ptrunc, E3, tot_var,
     tot_var_eig, W_all, 
     standard_factor,
     var_expl_by_retained] = limkb.step1_compress_individual_var(X_var, var, ntrunc, nmodes_sic, 
                                                                 var_dict1, areawt=acell_1d,
                                                                  wt=wt, sic_separate=True)
    
    v = {}
    var_dict = {}
    v['lat'] = var_dict1[var]['lat']
    v['lon'] = var_dict1[var]['lon']
    v['var_ndof'] = var_dict1[var]['var_ndof']
    v['time'] = np.concatenate((var_dict1[var]['time'], var_dict2[var]['time']))
    v['climo'] = np.concatenate((var_dict1[var]['climo'], var_dict2[var]['climo']))

    var_dict[var] = v

Changing units of ds1 sic be a between 0 to 1
Changing units of ds2 sic be a between 0 to 1
changing cellarea units from centimeter^2 to km^2
truncating to 50


MemoryError: Unable to allocate 22.0 GiB for an array with shape (122880, 24000) and data type float64

In [ ]:
mod_data_trunc = {}

mod_data_trunc['var_dict1'] = var_dict1
mod_data_trunc['var_dict2'] = var_dict2
mod_data_trunc['var_dict'] = var_dict

mod_data_trunc['Ptrunc'] = Ptrunc
mod_data_trunc['E3'] = E3
mod_data_trunc['standard_factor'] = standard_factor
mod_data_trunc['W_all'] = W_all

In [ ]:
if 'datetime64' in str(type(var_dict[var]['time'][0])):
    start_yr = str(var_dict[var]['time'][0].astype('M8[Y]'))
    end_yr = str(var_dict[var]['time'][-1].astype('M8[Y]'))
else: 
    start_yr = str(var_dict[var]['time'][0].year)
    end_yr = str(var_dict[var]['time'][-1].year)

mod_folder = '/home/disk/p/mkb22/Documents/si_analysis_kb/LIMs/SI_LIMs/truncated_model_data/'

if var is 'sic':
    nmod = nmodes_sic
else: 
    nmod = nmodes
    
mod_filename = (var+'_ntrunc'+str(nmod)+'_month'+str(mo)+'_'+str(train_dsource1)+'_'+str(train_dsource2)[-6:]+
                '_ntrain_'+start_yr+'_'+end_yr+'_'+today_date+'.pkl')

print('would save in: '+mod_folder+mod_filename)

In [ ]:
print('saving in: '+mod_folder+mod_filename)
pickle.dump(mod_data_trunc, open(mod_folder+mod_filename, "wb" ) )